In [1]:
import pandas as pd
import xarray as xr
import os
import glob

##################################################提取文件路径##########################################################
def get_file_paths(folder_path1, folder_path2):
    # 使用 glob 模块获取第一个文件夹中所有符合条件的文件路径
    file_paths0 = glob.glob(os.path.join(folder_path1, '*', '*_normal.nc'))
    
    # 使用 glob 模块获取第二个文件夹中所有符合条件的文件路径
    file_paths1 = glob.glob(os.path.join(folder_path2, '*', '*_physical.nc'))
    
    return file_paths0, file_paths1

In [2]:
folder_path0 = '/exports/WRF-CHEM/model/wrfpost/normal_new/'# 要提取的nc文件路径
folder_path1 = '/exports/WRF-CHEM/model/wrfpost/physical_new'# 要提取的nc文件路径

file_paths0, file_paths1 = get_file_paths(folder_path0, folder_path1)

variable_names0 = ['pressure','height','o3','no2','so2','hcho','pm25','tempc','ua','va','pblh']  # 要提取的变量名称列表
variable_names1 = ['rh']  # 要提取的变量名称列表

lat_range = [26,35]  # 纬度范围
lon_range = [116,125]  # 经度范围
#长江三角洲地区的地理范围介于，东经118°20′一123°25′，北纬28°45′～33°25′

In [3]:
timedt=pd.date_range('20230801','20230901')
t=timedt[0]

In [4]:
L0=[glob.glob(folder_path0+f'/{t.strftime("%YM%m")}*/{t.strftime("%Y%m%d")}*')[0] for t in timedt]
L1=[glob.glob(folder_path1+f'/{t.strftime("%YM%m")}*/{t.strftime("%Y%m%d")}*')[0] for t in timedt]
L0
L1

['/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023080100_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023080200_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023080300_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023080400_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023080500_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023080600_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023080700_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023080800_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023080900_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023081000_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023081100_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physical_new/2023M08/2023081200_physical.nc',
 '/exports/WRF-CHEM/model/wrfpost/physic

In [5]:
for [l0,l1] in zip(L0,L1):
    print(l0)

/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023080100_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023080200_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023080300_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023080400_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023080500_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023080600_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023080700_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023080800_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023080900_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023081000_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023081100_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023081200_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023081300_normal.nc
/exports/WRF-CHEM/model/wrfpost/normal_new/2023M08/2023081400_no

In [6]:
ds0=xr.open_dataset(l0)
ds1=xr.open_dataset(l1)

# ds0=xr.open_dataset('/exports/WRF-CHEM/model/wrfpost/normal_new/2023M06/2023060100_normal.nc')
# ds1=xr.open_dataset('/exports/WRF-CHEM/model/wrfpost/physical_new/2023M06/2023060100_physical.nc')

lat = ds0['xlat']
lon = ds0['xlon']
# 筛选经纬度范围
lat_indices=(lat >=lat_range[0])&(lat<= lat_range[1])
lon_indices =(lon>=lon_range[0])&(lon<= lon_range[1])

p_indices=lat_indices&lon_indices



###############################variable_names1

variable = ds1[variable_names1]
# 插值计算
interp_variable = variable.interp(Time=[2.5, 5.5], method='linear')
# 限定经纬度
interp_data = interp_variable[variable_names1].where(p_indices)  ###

data1=interp_data.mean(dim=('south_north','west_east'))



###############################variable_names0
variable  = ds0[variable_names0]

# 插值计算
interp_variable = variable.interp(Time=[2.5, 5.5], method='linear')

# 限定经纬度
interp_data = interp_variable[variable_names0].where(p_indices)
data0=interp_data.mean(dim=('south_north','west_east'))
Data=xr.merge([data0[variable_names0],data1[variable_names1]])

Data=Data.sel(bottom_top=Data.pressure[0]>100)  #选择100hPa以下的对流层

In [7]:

import numpy as np
np.abs(Data.pressure.diff(dim='bottom_top'))
mm_air=28.97/1000
mm_o3=3*16.0/1000
NA=6.02e23
gearth=9.81

(Data.no2[:,:-1]*np.abs((Data.pressure*100).diff(dim='bottom_top'))*NA/gearth/mm_air/1e6*1e-4#*mm_o3
    ).sum(dim='bottom_top')



<xarray.DataArray (Time: 2)>
array([2.81148597e+15, 1.96508280e+15])
Dimensions without coordinates: Time

In [8]:
#对流层 取气压到100hPa

Data['no2_tc']=(Data.no2[:,:-1]*np.abs((Data.pressure*100).diff(dim='bottom_top'))*NA/gearth/mm_air/1e6*1e-4#*mm_o3
    ).sum(dim='bottom_top')

Data['hcho_tc']=(Data.hcho[:,:-1]*np.abs((Data.pressure*100).diff(dim='bottom_top'))*NA/gearth/mm_air/1e6*1e-4#*mm_o3
    ).sum(dim='bottom_top')
Data['o3_tc']=(Data.o3[:,:-1]*np.abs((Data.pressure*100).diff(dim='bottom_top'))*NA/gearth/mm_air/1e6*1e-4#*mm_o3
    ).sum(dim='bottom_top')

Data['so2_tc']=(Data.so2[:,:-1]*np.abs((Data.pressure*100).diff(dim='bottom_top'))*NA/gearth/mm_air/1e6*1e-4#*mm_o3
    ).sum(dim='bottom_top')

In [9]:
Data

<xarray.Dataset>
Dimensions:   (Time: 2, bottom_top: 36)
Dimensions without coordinates: Time, bottom_top
Data variables: (12/16)
    pressure  (Time, bottom_top) float64 996.5 993.2 989.8 ... 135.3 119.3 105.6
    height    (Time, bottom_top) float64 12.78 42.61 ... 1.565e+04 1.637e+04
    o3        (Time, bottom_top) float64 0.0452 0.04573 ... 0.1392 0.2017
    no2       (Time, bottom_top) float64 0.00162 0.001568 ... 5.093e-05
    so2       (Time, bottom_top) float64 0.0005625 0.0005658 ... 1.508e-05
    hcho      (Time, bottom_top) float64 0.001215 0.00123 ... 1.905e-05
    ...        ...
    pblh      (Time) float64 627.7 760.4
    rh        (Time, bottom_top) float64 78.76 79.12 79.91 ... 24.51 22.73 21.03
    no2_tc    (Time) float64 2.811e+15 1.965e+15
    hcho_tc   (Time) float64 6.566e+15 6.198e+15
    o3_tc     (Time) float64 1.062e+18 1.053e+18
    so2_tc    (Time) float64 1.779e+15 1.829e+15

In [10]:
p_850_id=np.abs(Data.pressure - 850).argmin(dim='bottom_top')
#Data=Data.sel(bottom_top=p_850_id)

In [11]:
list(Data.variables.keys())

['pressure',
 'height',
 'o3',
 'no2',
 'so2',
 'hcho',
 'pm25',
 'tempc',
 'ua',
 'va',
 'pblh',
 'rh',
 'no2_tc',
 'hcho_tc',
 'o3_tc',
 'so2_tc']

In [12]:
Data[['rh','ua','va','tempc']].sel(bottom_top=p_850_id)

<xarray.Dataset>
Dimensions:  (Time: 2)
Dimensions without coordinates: Time
Data variables:
    rh       (Time) float64 55.81 63.67
    ua       (Time) float64 -6.794 -6.677
    va       (Time) float64 -3.7 -3.137
    tempc    (Time) float64 18.65 18.62

In [13]:


Data[['rh','ua','va','tempc']]=Data[['rh','ua','va','tempc']].sel(bottom_top=p_850_id)

In [14]:
Data[['o3',
 'no2',
 'so2',
 'hcho',
 'pm25',]]=Data[['o3',
 'no2',
 'so2',
 'hcho',
 'pm25',]].sel(bottom_top=0)

In [15]:
Data[['o3',
 'no2',
 'so2',
 'hcho',
 'pm25',
 'tempc',
 'ua',
 'va',
 'pblh',
 'rh',
 'no2_tc',
 'hcho_tc',
 'o3_tc',
 'so2_tc']]

<xarray.Dataset>
Dimensions:  (Time: 2)
Dimensions without coordinates: Time
Data variables: (12/14)
    o3       (Time) float64 0.0452 0.04935
    no2      (Time) float64 0.00162 0.0009799
    so2      (Time) float64 0.0005625 0.0004578
    hcho     (Time) float64 0.001215 0.0009212
    pm25     (Time) float64 10.64 9.694
    tempc    (Time) float64 18.65 18.62
    ...       ...
    pblh     (Time) float64 627.7 760.4
    rh       (Time) float64 55.81 63.67
    no2_tc   (Time) float64 2.811e+15 1.965e+15
    hcho_tc  (Time) float64 6.566e+15 6.198e+15
    o3_tc    (Time) float64 1.062e+18 1.053e+18
    so2_tc   (Time) float64 1.779e+15 1.829e+15

In [16]:
Data['Time']=[2.5, 5.5]

In [17]:
Data2=Data[['o3',
 'no2',
 'so2',
 'hcho',
 'pm25',
 'tempc',
 'ua',
 'va',
 'pblh',
 'rh',
 'no2_tc',
 'hcho_tc',
 'o3_tc',
 'so2_tc']]
Data2['date']=[t.strftime("%Y%m%d")]
Data2.to_dataframe()

,,o3,no2,so2,hcho,pm25,tempc,ua,va,pblh,rh,no2_tc,hcho_tc,o3_tc,so2_tc
Time,date,,,,,,,,,,,,,,
2.5,20230801,0.045196,0.00162,0.000563,0.001215,10.641902,18.651282,-6.794036,-3.700398,627.666433,55.805102,2.811486e+15,6.565817e+15,1.061507e+18,1.779015e+15
5.5,20230801,0.049352,0.00098,0.000458,0.000921,9.694344,18.619308,-6.676690,-3.136941,760.404688,63.665240,1.965083e+15,6.198300e+15,1.052791e+18,1.828805e+15


In [18]:
df=Data2.to_dataframe().reset_index()


In [19]:
fname='shanghai_'+t.strftime("%Y%m%d")+'.csv'
fname

'shanghai_20230801.csv'

In [49]:
df.to_csv(fname)